# Movie Recommender Models

In [2]:
pip install contractions

     |████████████████████████████████| 284 kB 4.1 MB/s 
     |████████████████████████████████| 321 kB 33.9 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85445 sha256=084abdd77f45bb32c2f89ec1b2249677f8b9376178ba40dc644b4a0935792f4a
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [3]:
import pandas as pd
import nltk
import re
import numpy as np
import contractions
from datetime import date

## Preparing the Corpus

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Reading in the data and basic cleaning
colab_old_movies = "/content/drive/MyDrive/Colab_Notebooks/cineman_streamlit_app/tmdb_5000_movies.csv"
colab_recent_movies = "/content/drive/MyDrive/Colab_Notebooks/cineman_streamlit_app/2021-10-29_zurich_movie_overviews.csv"

local_old_movies = "../data/external/tmdb_5000_movies.csv"
#local_recent_movies = f"../data/raw/{date.today()}_zurich_movie_overviews.csv"
local_recent_movies = f"../data/raw/2021-10-29_zurich_movie_overviews.csv"

movie_reviews_old = pd.read_csv(local_old_movies)
movie_reviews_old = movie_reviews_old.sort_values("popularity", ascending=False).drop_duplicates("original_title").reset_index()

movie_reviews_recent = pd.read_csv(local_recent_movies)

In [38]:
def prepare_movie_descriptions(df):
    df_2 = df.copy()
    df_2 = df_2.loc[df_2["overview"].notna()]
    df_2["tagline"] = df_2["tagline"].fillna("")
    df_2["description"] = df_2["tagline"] + " " + df_2["overview"]
    df_2 = df_2[["original_title", "description"]]

    return df_2

In [54]:
movie_reviews_old_2 = prepare_movie_descriptions(movie_reviews_old)
print(movie_reviews_old_2.shape)
movie_reviews_old_2.head(3)

(4798, 2)


,original_title,description
0,Minions,"Before Gru, they had a history of bad bosses M..."
1,Interstellar,Mankind was born on Earth. It was never meant ...
2,Deadpool,Witness the beginning of a happy ending Deadpo...


In [55]:
movie_reviews_recent_2 = prepare_movie_descriptions(movie_reviews_recent)
print(movie_reviews_recent_2.shape)
movie_reviews_recent_2.head(3)

(37, 2)


,original_title,description
0,Wild,A woman with a tragic past decides to start h...
1,Azor,"Yvan De Wiel, a private banker from Geneva, i..."
2,"Quo Vadis, Aida?","Bosnia, July 1995. Aida is a translator for t..."


In [51]:
movie_reviews_df = pd.concat([movie_reviews_old_2, movie_reviews_recent_2]
                            ).drop_duplicates("original_title", keep="last").reset_index(drop=True)

movie_reviews_df.shape

(4832, 2)

In [67]:
# Clean and normalize the text

nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # remove special characters
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, flags=re.I|re.A)
    # lower case
    doc = doc.lower()
    # strip whitespaces
    doc = doc.strip()
    # fix contractions
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(movie_reviews_df['description']))
print(len(norm_corpus))
norm_corpus[0:3]

[nltk_data] Downloading package stopwords to /home/angela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/angela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


4832


array(['gru history bad bosses minions stuart kevin bob recruited scarlet overkill supervillain alongside inventor husband herb hatches plot take world',
       'mankind born earth never meant die interstellar chronicles adventures group explorers make use newly discovered wormhole surpass limitations human space travel conquer vast distances involved interstellar voyage',
       'witness beginning happy ending deadpool tells origin story former special forces operative turned mercenary wade wilson subjected rogue experiment leaves accelerated healing powers adopts alter ego deadpool armed new abilities dark twisted sense humor deadpool hunts man nearly destroyed life'],
      dtype='<U797')

In [66]:
tokenized_docs = [doc.split() for doc in norm_corpus]
tokenized_docs[0]

['gru',
 'history',
 'bad',
 'bosses',
 'minions',
 'stuart',
 'kevin',
 'bob',
 'recruited',
 'scarlet',
 'overkill',
 'supervillain',
 'alongside',
 'inventor',
 'husband',
 'herb',
 'hatches',
 'plot',
 'take',
 'world']

## Fast Text Model

In [61]:
#from gensim.models import FastText

# ideal config params size: 300, window: 30, min_count=2 or more, iter=50 or more (use 10 if it takes too much time)
#ft_model = FastText(tokenized_docs, size=300, window=30, min_count=2, workers=4, sg=1, iter=50)

In [60]:
import pickle

#colab_modelpath = '/content/drive/MyDrive/Colab_Notebooks/cineman_streamlit_app/fast_text_model.sav'
#pickle.dump(ft_model, open(colab_modelpath, 'wb'))

local_modelpath = "../models/fast_text_model.sav"

ft_model = pickle.load(open(local_modelpath, 'rb'))

In [62]:
# Turning word level embeddings into document embeddings

def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [68]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 300)
doc_vecs_ft.shape

AttributeError: The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4797,4798,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808,4809,4810,4811,4812,4813,4814,4815,4816,4817,4818,4819,4820,4821,4822,4823,4824,4825,4826,4827,4828,4829,4830,4831,4832,4833,4834,4835,4836
0,1.000000,0.544371,0.581542,0.592283,0.592020,0.516476,0.551647,0.594866,0.480950,0.556415,0.504953,0.544554,0.450361,0.497972,0.575938,0.514532,0.542451,0.526235,0.558689,0.546900,0.531221,0.487438,0.508384,0.468716,0.517595,0.494261,0.598329,0.663213,0.429199,0.576758,0.518754,0.558232,0.538881,0.488769,0.462096,0.581429,0.652049,0.433584,0.525556,0.522751,...,0.558603,0.507487,0.505278,0.504545,0.524787,0.576169,0.484424,0.530920,0.465747,0.494652,0.593491,0.521854,0.573608,0.462314,0.560031,0.438122,0.446248,0.526923,0.477657,0.451765,0.459440,0.505658,0.525589,0.573770,0.515914,0.484607,0.512085,0.500587,0.552740,0.450980,0.524836,0.463800,0.520003,0.468681,0.511174,0.580674,0.484563,0.458714,0.478598,0.461432
1,0.544371,1.000000,0.537805,0.527232,0.632371,0.524081,0.562125,0.603941,0.592666,0.562151,0.517727,0.549456,0.600800,0.565233,0.627764,0.625662,0.603222,0.619507,0.620828,0.557848,0.595318,0.566820,0.572894,0.595755,0.558125,0.622211,0.609759,0.641452,0.528239,0.562032,0.567556,0.554760,0.575576,0.487617,0.513719,0.562258,0.584946,0.613008,0.584761,0.546790,...,0.607860,0.564015,0.542341,0.520468,0.558336,0.534277,0.552335,0.528620,0.584298,0.630011,0.553508,0.537334,0.609757,0.528597,0.648283,0.611970,0.586956,0.520292,0.576215,0.535664,0.523466,0.568320,0.649052,0.579829,0.568512,0.548104,0.563423,0.586294,0.568060,0.520481,0.579760,0.542795,0.630653,0.528651,0.585672,0.535563,0.568664,0.478731,0.568754,0.550561
2,0.581542,0.537805,1.000000,0.598782,0.515564,0.542140,0.546884,0.597189,0.600254,0.574654,0.530482,0.724822,0.536980,0.557343,0.580439,0.542110,0.600311,0.591538,0.643132,0.574130,0.593517,0.536523,0.559324,0.562836,0.513675,0.490106,0.572437,0.564808,0.503412,0.709779,0.577979,0.568627,0.597522,0.545156,0.487478,0.685331,0.618102,0.525976,0.637470,0.570330,...,0.563936,0.539690,0.537821,0.632863,0.555518,0.544320,0.516927,0.488362,0.534422,0.518529,0.698784,0.539263,0.544275,0.461307,0.505210,0.500074,0.528395,0.537062,0.514364,0.666500,0.510037,0.502717,0.622322,0.535559,0.508078,0.539632,0.551787,0.512650,0.548212,0.457949,0.561780,0.493700,0.616689,0.528419,0.502147,0.548603,0.544229,0.499351,0.523645,0.509130
3,0.592283,0.527232,0.598782,1.000000,0.555343,0.533218,0.517675,0.591228,0.578659,0.700445,0.523377,0.589652,0.482125,0.559685,0.558481,0.540689,0.531326,0.526509,0.591869,0.528313,0.550357,0.551137,0.506167,0.492040,0.552159,0.525503,0.573395,0.575588,0.453515,0.603641,0.546870,0.608456,0.603111,0.526897,0.473249,0.565594,0.622762,0.502130,0.580700,0.541185,...,0.626967,0.565390,0.528661,0.490101,0.544845,0.526291,0.540198,0.484673,0.534362,0.516468,0.565392,0.538312,0.518767,0.535217,0.531851,0.472914,0.520104,0.601663,0.477619,0.534564,0.526969,0.560517,0.597641,0.529527,0.488080,0.517243,0.486047,0.584333,0.528291,0.432248,0.611437,0.491963,0.528534,0.556642,0.575201,0.496025,0.505667,0.522671,0.477358,0.526365
4,0.592020,0.632371,0.515564,0.555343,1.000000,0.494332,0.545823,0.624732,0.528614,0.579878,0.589968,0.548142,0.541237,0.593001,0.590801,0.545613,0.639995,0.588601,0.599919,0.566544,0.567935,0.524994,0.531374,0.513775,0.570021,0.565555,0.650608,0.679214,0.517620,0.572972,0.520335,0.550773,0.549041,0.570075,0.486257,0.557744,0.629565,0.449697,0.522988,0.575440,...,0.570471,0.512469,0.534640,0.513892,0.528891,0.502538,0.484556,0.480833,0.500821,0.542159,0.589421,0.520998,0.634105,0.451998,0.590846,0.552599,0.467333,0.492715,0.553626,0.545445,0.533749,0.486529,0.602845,0.519703,0.555800,0.522062,0.534931,0.514907,0.539751,0.421690,0.541900,0.601287,0.561175,0.564819,0.520212,0.585266,0.562736,0.496714,0.555030,0.499221


In [86]:
# What are the indices of recent movies in the overall df?
recent_movies = movie_reviews_recent_2["original_title"].values
recent_movie_idx = movie_reviews_df[movie_reviews_df["original_title"].isin(recent_movies)].index

In [87]:
all_movies = movie_reviews_df['original_title'].values

In [88]:
np.where(all_movies == movie_title)

(array([29]),)

In [71]:
recent_similarities = movie_similarities[recent_movie_idx]
recent_similarities

array([0.64390206, 0.53778644, 0.58092955, 0.57675754, 0.56203207,
       0.70977915, 0.60364094, 0.57297247, 0.53233894, 0.54355942,
       0.59399121, 0.58231107, 0.58427417, 0.55879323, 0.69270922,
       0.50896647, 0.51263898, 0.56717083, 0.55415708, 0.67525287,
       0.55582693, 0.5595249 , 0.52451174, 0.5581815 , 0.5048028 ,
       0.55943185, 0.52463726, 0.59748187, 0.56088283, 1.        ,
       0.55876324, 0.55576169, 0.56888847, 0.60496752, 0.51547455,
       0.55950282, 0.5494904 , 0.49864852, 0.57207141, 0.56117958])

In [80]:
len(similar_movies), len(similar_recent_movies)

(4837, 40)

In [82]:
doc_sim_df.iloc[29][29]

1.000000000000001

In [111]:
movie_reviews_old_2[movie_reviews_old_2["original_title"] == "Die Hard"].iloc[:,1].values

array(["40 Stories. Twelve Terrorists. One Cop. NYPD cop, John McClane's plan to reconcile with his estranged wife is thrown for a serious loop when minutes after he arrives at her office, the entire building is overtaken by a group of terrorists. With little help from the LAPD, wisecracking McClane sets out to single-handedly rescue the hostages and bring the bad guys down."],
      dtype=object)

In [121]:
movie_reviews_recent_2[movie_reviews_recent_2["original_title"] == "Dune"].iloc[:,1].values

array(["Beyond fear, destiny awaits. Paul Atreides, a brilliant and gifted young man born into a great destiny beyond his understanding, must travel to the most dangerous planet in the universe to ensure the future of his family and his people. As malevolent forces explode into conflict over the planet's exclusive supply of the most precious resource in existence-a commodity capable of unlocking humanity's greatest potential-only those who can conquer their fear will survive."],
      dtype=object)

In [120]:
movie_title = "The Lord of the Rings: The Fellowship of the Ring"
movie_idx = np.where(all_movies == movie_title)[0][0]
movie_idx
movie_similarities = doc_sim_df.iloc[movie_idx].values
#recent_similarities = movie_similarities[recent_movie_idx]
similar_movies = np.argsort(-movie_similarities)
similar_recent_movies = [index for index in similar_movies if index in recent_movie_idx]

similar_recent_movies

for i in similar_recent_movies:
  print(all_movies[i])

Dune
Dune
The Mack
Halloween Kills
The Harder They Come
Shang-Chi and the Legend of the Ten Rings
Supernova
Ghosts
Contra
The Ice Road
After We Fell
Cry Macho
Ron's Gone Wrong
Space Jam: A New Legacy
Supernova
The Boss Baby: Family Business
Die Schule der magischen Tiere
No Time to Die
Tom & Jerry
Wild
Wild
Schachnovelle
Quo Vadis, Aida?
The Last Duel
Wesele
Petite maman
Es ist nur eine Phase, Hase
Glassboy
Lamb
Und morgen seid ihr tot
Titane
Azor
Style Wars 2
Venom: Let There Be Carnage
The Father
Nomadland
Herr Bachmann und seine Klasse
The French Dispatch
Monte Verità
Bekenntnisse des Hochstaplers Felix Krull


In [73]:
recent_similarities

array([0.64390206, 0.53778644, 0.58092955, 0.57675754, 0.56203207,
       0.70977915, 0.60364094, 0.57297247, 0.53233894, 0.54355942,
       0.59399121, 0.58231107, 0.58427417, 0.55879323, 0.69270922,
       0.50896647, 0.51263898, 0.56717083, 0.55415708, 0.67525287,
       0.55582693, 0.5595249 , 0.52451174, 0.5581815 , 0.5048028 ,
       0.55943185, 0.52463726, 0.59748187, 0.56088283, 1.        ,
       0.55876324, 0.55576169, 0.56888847, 0.60496752, 0.51547455,
       0.55950282, 0.5494904 , 0.49864852, 0.57207141, 0.56117958])

In [51]:
all_movies = movie_reviews_df['original_title'].values
recent_movies = movie_reviews_recent_2["original_title"].values

def movie_recommender(movie_title, all_movies=all_movies, recent_movies=recent_movies, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(all_movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities[recent_movie_idx])[1:6]
    # get top 5 movies
    similar_movies = all_movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [ ]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [52]:

#movies_list, movies_list.shape

popular_movies = ["Skyfall"]

for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, all_movies=all_movies, recent_movies=recent_movies, doc_sims=doc_sim_df))
    print()

Movie: Skyfall
Top 5 recommended Movies: ['Spider-Man 3' 'Man of Steel' 'The Hobbit: The Battle of the Five Armies'
 'Avatar' 'X-Men: The Last Stand']

